In [2]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy.sparse import coo_array
from scipy import sparse
from cdlib import algorithms
from cdlib import evaluation
import sklearn
from utils import *
from distances import *
from consensus import *
import math
import itertools
import random
import time
from pathlib import Path

Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw'}


In [1]:
cons_name = "v4"

In [54]:
a = np.array([1,2,3,4])
b = np.array([1,3,3,4])
c = np.equal(a, b)
d = np.sum(c)
print(c)
print(d)

[ True False  True  True]
3


In [63]:
# DeltaSOD is calculated following the paper titled "Integrating Microarray Data by Consensus Clustering"
# by Filkov and Skiena
# Assumes the elements of the cluster are named as 0-based indices
def v4_consensus(P_list, niter=10, starting_partition=None, verbose=False):
    G = nx.Graph(P_list[0]["graph"])
    n = len(list(G.nodes()))
    k = len(P_list)
    A = nx.to_numpy_array(G)
    nz_rows, nz_cols = np.nonzero(A)
    
    t1 = time.time()
    P_list_asn = []
    c = np.zeros((n,k))
    for i in range(k):
        clust_lst = P_list[i]["partition"]
        clust_asn = clust_lst_to_asn(clust_lst)
        c[:,i] = np.array(clust_asn)
    t2 = time.time()
    print("Time to generate cluster assignment matrix:", t2-t1)
    
    t1 = time.time()
    Aw = np.array(A)
    nz_elems = []
    for i in range(len(nz_rows)):
        Aw[nz_rows[i], nz_cols[i]] = np.sum( c[nz_rows[i],:] == c[nz_cols[i],:] )
        Aw[nz_cols[i], nz_rows[i]] = Aw[nz_rows[i], nz_cols[i]]
        nz_elems.append((nz_rows[i], nz_cols[i], Aw[nz_rows[i], nz_cols[i]]))
    Gw = nx.from_numpy_array(Aw)
    nz_elems = sorted(nz_elems, key=lambda x: x[2], reverse=True)
    t2 = time.time()
    print("Time to generate weighted consensus graph:", t2-t1)
    
    
    t1 = time.time()
    nodes = list(Gw.nodes())
    neighbors = {}
    bfs_neighbors = {}
    for node in nodes:
        neighbors[node] = list(Gw.neighbors(node))
        bfs_neighbors[node] = []
        bfs_edges = list(nx.bfs_tree(Gw, source=node, depth_limit=3).edges())
        for edge in bfs_edges:
            y = edge[1]
            bfs_neighbors[node].append(y)
        #bfs_neighbors[node] = set(bfs_neighbors[node])
    t2 = time.time()
    print("Neighborhood list prep:", t2-t1)
    
    """
    t1 = time.time()
    row = []
    col = []
    val = []
    for x in P_list:
        graph = x["graph"]
        partition = x["partition"]
        for cluster in partition:
            for i in range(len(cluster)):
                for j in range(i+1, len(cluster)):
                    item_1 = cluster[i]
                    item_2 = cluster[j]
                    row.append(min(int(item_1), int(item_2)))
                    col.append(max(int(item_1), int(item_2)))
                    val.append(int(1))
                    
    r = coo_array((val, (row, col)), shape=(n, n))
    print("r.nnz:", r.nnz)
    t2 = time.time()
    print("Time to prepare r:", t2-t1)
    """
    
    t1 = time.time()
    row = []
    col = []
    val = []
    for x in P_list:
        graph = x["graph"]
        partition_lst = x["partition"]
        partition_asn = clust_lst_to_asn(partition_lst)
        for node in nodes:
            for bfsn in bfs_neighbors[node]:
                if partition_asn[node] == partition_asn[bfsn]:
                    row.append(node)
                    col.append(bfsn)
                    val.append(1)
                    
                    #row.append(bfsn)
                    #col.append(node)
                    #val.append(1)
    r = coo_array((val, (row, col)), shape=(n, n))
    print("r.nnz:", r.nnz)
    t2 = time.time()
    print("Time to prepare r:", t2-t1)
    
    r = r.tocsr()
    R = r.sum()
    if verbose:
        print("R:", R)
    
    t1 = time.time()
    rDense = r.toarray() # Should be upper triangular
    rDense = rDense + rDense.T # Making symmetric
    t2 = time.time()
    print("Time to prepare rDense:", t2-t1)
    
    K = k - 2 * rDense
    np.fill_diagonal(K, -k) # Adding diagonal entries
    
    t1 = time.time()
    refined_partition = None
    if starting_partition:
        refined_partition = list(starting_partition)
    else:
        refined_partition = []
        for i in range(n):
            refined_partition.append([str(i)])
    
    refined_partition_map = clust_lst_to_map(refined_partition)
    t2 = time.time()
    print("Time to initialize:", t2-t1)
    
    t1 = time.time()
    items = list(refined_partition_map.keys())
    M = np.zeros((n, len(refined_partition)))
    for item_1 in items:
        for partition_id in range(len(refined_partition)):
            for item_2 in refined_partition[partition_id]:
                if(item_1 != item_2):
                    M[int(item_1),partition_id] = M[int(item_1),partition_id] + K[int(item_1), int(item_2)]
    
    mv = np.min(M, axis=1)
    mb = np.argmin(M, axis=1)
    t2 = time.time()
    print("Time to prepare M:", t2-t1)
    
    tSearch = 0
    tUpdate = 0
    tMovement = 0
    count = 0
    it = 1
    while(it <= niter):
        opt_item = items[0]
        opt_deltaS = 0
        opt_a = refined_partition_map[items[0]]
        opt_b = refined_partition_map[items[0]]
        opt_x = int(opt_item)
        flag = False
        for i in range(len(nz_elems)):
            if verbose:
                print("nz_elems[", i, "]", nz_elems[i])
            t1 = time.time()
            x1 = nz_elems[i][0]
            x2 = nz_elems[i][1]
            a1 = refined_partition_map[str(x1)]
            a2 = refined_partition_map[str(x2)]
            deltaS1 = M[x1,a2] - M[x1,a1]
            deltaS2 = M[x2,a1] - M[x2,a2]
            x = None
            item = None
            b = None
            deltaS = None
            if (deltaS1 < deltaS2):
                x = x1
                item = str(x1)
                a = a1
                b = a2
                deltaS = deltaS1
            else:
                x = x2
                item = str(x2)
                a = a2
                b = a1
                deltaS = deltaS2
            t2 = time.time()
            tSearch = tSearch + t2-t1
            if (deltaS is not None) and (deltaS < 0) and (a != b):
                opt_item = item
                opt_deltaS = deltaS
                opt_a = a
                opt_b = b
                opt_x = x
                
                if verbose:
                    print("---")
                    print("Move Count:", count+1, "Optimum move results in", opt_deltaS)
                    print("Move:", opt_item)
                    print("From", opt_a, ":", refined_partition[opt_a])
                    print("To", opt_b, ":", refined_partition[opt_b])
                    
                t1 = time.time()
                #print(len(items), len(list(nx.bfs_tree(Gw, source=opt_x, depth_limit=5).edges())))
                #bfs_edges = list(nx.bfs_tree(Gw, source=opt_x, depth_limit=5).edges())
                #for edge in bfs_edges:
                    #y = edge[1]
                for y_str in bfs_neighbors[opt_x]:
                    y = int(y_str)
                    if y != opt_x:
                        M[y, opt_a] = M[y, opt_a] - K[y, opt_x]
                        M[y, opt_b] = M[y, opt_b] + K[y, opt_x]
                t2 = time.time()
                tUpdate = tUpdate + (t2-t1)
                
                t1 = time.time()
                refined_partition[opt_a].remove(opt_item)
                refined_partition[opt_b].append(opt_item)
                refined_partition_map[opt_item] = opt_b
                t2 = time.time()
                tMovement = tMovement + (t2-t1)
                if verbose:
                    print("---")
            
                count = count + 1
                flag = True
        if flag == False:
            break

        it = it + 1
    print("Time to search moves:", tSearch)
    print("Time to update M:", tUpdate)
    
    t1 = time.time()
    empty_clusters = []
    for i in range(len(refined_partition)):
        if len(refined_partition[i]) == 0:
            empty_clusters.append(i)
            
    empty_clusters.sort(reverse=True)
    for e in empty_clusters:
        del refined_partition[e]
    t2 = time.time()
    print("Time to delete empty partitions:", t2-t1)
        
    #G = nx.from_scipy_sparse_array(r)
    
    return {"graph": nx.Graph(Gw), "partition": list(refined_partition)}

# n=200

In [64]:
n = 200
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [65]:
import random

n = 200
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v4_consensus(P_list, niter=1000, starting_partition = None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)

LFR/n200/LFR_n200_mu01_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.0024747848510742188
Time to generate weighted consensus graph: 0.01470184326171875
Neighborhood list prep: 0.062403202056884766
r.nnz: 235796
Time to prepare r: 0.09558224678039551
Time to prepare rDense: 8.273124694824219e-05
Time to initialize: 0.0022764205932617188
Time to prepare M: 0.03168153762817383
Time to search moves: 0.00482940673828125
Time to update M: 0.024632930755615234
Time to delete empty partitions: 6.151199340820312e-05
mu 1 , number of clusters 16
Time: 0.2635152339935303
LFR/n200/LFR_n200_mu02_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.0030477046966552734
Time to generate weighted consensus graph: 0.014647722244262695
Neighborhood list prep: 0.07484197616577148
r.nnz: 227548
Time to prepare r: 0.10245895385742188
Time to prepare rDense: 8.630752563476562e-05
Time to initialize: 7.2479248046875e-05
Time to prepare M: 0.030550241470336914
Time to search move

# n=1000

In [66]:
n = 1000
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [67]:
import random

n = 1000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v4_consensus(P_list, niter=1000, starting_partition=None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)
                

LFR/n1000/LFR_n1000_mu01_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.012668609619140625
Time to generate weighted consensus graph: 0.14499902725219727
Neighborhood list prep: 1.0340590476989746
r.nnz: 1377994
Time to prepare r: 1.0513384342193604
Time to prepare rDense: 0.0022389888763427734
Time to initialize: 0.012945413589477539
Time to prepare M: 0.8441145420074463
Time to search moves: 0.054007530212402344
Time to update M: 0.5379209518432617
Time to delete empty partitions: 0.0001800060272216797
mu 1 , number of clusters 38
Time: 3.8883891105651855
LFR/n1000/LFR_n1000_mu02_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.012375593185424805
Time to generate weighted consensus graph: 0.14670705795288086
Neighborhood list prep: 2.285011053085327
r.nnz: 2247534
Time to prepare r: 1.8389804363250732
Time to prepare rDense: 0.002635478973388672
Time to initialize: 0.020925283432006836
Time to prepare M: 0.857600212097168
Time to search moves: 0.0524

# n=5000

In [68]:
n = 5000
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [69]:
import random

n = 5000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v4_consensus(P_list, niter=1000, starting_partition = None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)
                

LFR/n5000/LFR_n5000_mu01_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.0655660629272461
Time to generate weighted consensus graph: 0.8906497955322266
Neighborhood list prep: 10.445287942886353
r.nnz: 10863926
Time to prepare r: 7.9341514110565186
Time to prepare rDense: 0.29502129554748535
Time to initialize: 0.0015158653259277344
Time to prepare M: 21.08752703666687
Time to search moves: 0.28795623779296875
Time to update M: 4.060292482376099
Time to delete empty partitions: 0.0008149147033691406
mu 1 , number of clusters 201
Time: 46.70052170753479
LFR/n5000/LFR_n5000_mu02_gamma30_beta11.mtx
Time to generate cluster assignment matrix: 0.06761717796325684
Time to generate weighted consensus graph: 0.9039409160614014
Neighborhood list prep: 23.66901135444641
r.nnz: 13876954
Time to prepare r: 14.251368761062622
Time to prepare rDense: 0.3007338047027588
Time to initialize: 0.14516496658325195
Time to prepare M: 21.25161051750183
Time to search moves: 0.43729305267333